In [ ]:
import json
import queue
import sounddevice as sd
import whisper
import numpy as np
import threading
import requests
import time
import wave
import os
from dotenv import load_dotenv
import openai
import soundfile as sf
import warnings
from collections import deque
import pyaudio
import keyboard
import re
import asyncio
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from fuzzywuzzy import fuzz
import pandas as pd


location_data = pd.read_excel("./data/기상청_단기예보_위경도.xlsx")

def convert_coordinates(row):
    """시분초 형식의 좌표를 십진수 도(degree) 형식으로 변환"""
    longitude = float(row['경도(시)']) + float(row['경도(분)'])/60 + float(row['경도(초)'])/3600
    latitude = float(row['위도(시)']) + float(row['위도(분)'])/60 + float(row['위도(초)'])/3600
    return longitude, latitude

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class ToolManager:
    def __init__(self):
        self.tools = {}

    def register_tool(self, name, tool):
        self.tools[name] = tool

    def get_tool_response(self, tool_name, *args, **kwargs):
        if tool_name in self.tools:
            return self.tools[tool_name].fetch_weather_by_location(*args, **kwargs)
        else:
            return "해당 도구를 찾을 수 없습니다."


class WeatherTool:
    def __init__(self):
        self.fetcher = WeatherFetcher()

    def fetch_weather_by_location(self, location_name):
        # 주어진 지역명에 대한 좌표 찾기
        coordinates = find_coordinates(location_name)
        if coordinates['위도'] is not None and coordinates['경도'] is not None:
            # 찾은 좌표를 사용하여 날씨 데이터 가져오기
            weather_data = self.fetcher.fetch_weather(nx=coordinates['경도'], ny=coordinates['위도'])
            return weather_data
        else:
            return None
         
tool_manager = ToolManager()
tool_manager.register_tool("weather", WeatherTool())


class LocationMatcher:
    def __init__(self, location_data):
        self.vectorizer = TfidfVectorizer(
            analyzer='char', 
            ngram_range=(1, 4),
            min_df=2,
            max_df=0.9
        )
        self.location_data = location_data
        self.locations = [
            f"{row['1단계']} {row['2단계']} {str(row['3단계'])}"
            for _, row in location_data.iterrows()
        ]
        self.vectors = self.vectorizer.fit_transform(self.locations)
        
    def find_coordinates(self, location_name):
        query_vec = self.vectorizer.transform([location_name])
        similarities = cosine_similarity(query_vec, self.vectors)[0]
        best_idx = np.argmax(similarities)
        
        row = self.location_data.iloc[best_idx]
        longitude, latitude = convert_coordinates(row)
        
        print(f"\n[DEBUG] 검색 위치: {location_name}")
        print(f"[DEBUG] 매칭 점수: {similarities[best_idx]:.2f}")
        print(f"[DEBUG] 매칭 위치: {self.locations[best_idx]}")
        print(f"[DEBUG] 격자 좌표: X={row['격자 X']}, Y={row['격자 Y']}\n")
        
        return {
            '입력 지명': location_name,
            '일치 지명': self.locations[best_idx],
            '일치율': similarities[best_idx] * 100,
            '위도': latitude,
            '경도': longitude,
            '격자X': row['격자 X'],
            '격자Y': row['격자 Y']
        }

# 사용 예시:

class WeatherFetcher:
    def __init__(self):
        self.base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
        self.api_key = os.getenv('KMA_API_KEY')
        self.default_x = 55
        self.default_y = 127

    def fetch_weather(self, date=None, time=None, nx=None, ny=None):
        if not date:
            date = datetime.now().strftime("%Y%m%d")
        if not time:
            time = "0500"

        params = {
            'serviceKey': self.api_key,
            'numOfRows': 10,
            'pageNo': 1,
            'dataType': 'JSON',
            'base_date': date,
            'base_time': time,
            'nx': nx if nx else self.default_x,
            'ny': ny if ny else self.default_y
        }
        
        print(f"\n[DEBUG] API 요청 파라미터:")
        print(f"URL: {self.base_url}")
        print(f"Parameters: {params}\n")

        try:
            response = requests.get(self.base_url, params=params)
            print(f"[DEBUG] API 응답 상태: {response.status_code}")
            if response.status_code == 200:
                data = response.json()
                print(f"[DEBUG] API 응답 데이터: {data}\n")
                return data
            else:
                print(f"[DEBUG] API 오류 응답: {response.text}\n")
                return None
        except Exception as e:
            print(f"[DEBUG] API 요청 실패: {str(e)}\n")
            return None

def get_weather_response(location_name):
    print(f"\n[DEBUG] 날씨 조회 시작: {location_name}")
    
    weather_tool = tool_manager.tools["weather"]
    
    matcher = LocationMatcher(location_data)
    find_coordinates = matcher.find_coordinates

    coordinates = find_coordinates(location_name)
    print(f"[DEBUG] 좌표 검색 결과: {coordinates}")
    
    weather_data = weather_tool.fetcher.fetch_weather(
        nx=coordinates['격자X'],
        ny=coordinates['격자Y']
    )
    
    if weather_data:
        items = weather_data['response']['body']['items']['item']
        weather_info = []
        print("\n[DEBUG] 날씨 정보 파싱:")
        location = coordinates["일치 지명"]
        weather_info.append(f"지역: {location}")
        for item in items:
            category = item['category']
            value = item['fcstValue']
            print(f"[DEBUG] 카테고리: {category}, 값: {value}")

            if category == 'TMP':
                weather_info.append(f"기온: {value}°C")
            elif category == 'SKY':
                sky_status = {"1": "맑음", "3": "구름많음", "4": "흐림"}.get(value, "알 수 없음")
                weather_info.append(f"하늘 상태: {sky_status}")
            elif category == 'PTY':
                rain_status = {"0": "없음", "1": "비", "2": "비/눈", "3": "눈"}.get(value, "알 수 없음")
                weather_info.append(f"강수 형태: {rain_status}")
        
        result = f"\n ".join(weather_info)
        print(f"\n[DEBUG] 최종 결과: {result}")
        return result
    else:
        print("\n[DEBUG] 날씨 데이터 없음")
        return "날씨 정보를 가져오는 데 실패했습니다."

In [53]:
get_weather_response( '서울 성수동')


[DEBUG] 날씨 조회 시작: 서울 성수동

[DEBUG] 검색 위치: 서울 성수동
[DEBUG] 매칭 점수: 0.47
[DEBUG] 매칭 위치: 서울 성동구 옥수동
[DEBUG] 격자 좌표: X=60, Y=126

[DEBUG] 좌표 검색 결과: {'입력 지명': '서울 성수동', '일치 지명': '서울 성동구 옥수동', '일치율': 46.83487905036402, '위도': 37.54371666666666, '경도': 127.0134638888889, '격자X': 60, '격자Y': 126}

[DEBUG] API 요청 파라미터:
URL: http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst
Parameters: {'serviceKey': 'g/+N4pQ1jAxfWQL13OIWZBhOALo3S7Wd7wj1joUg+BWtbu3Ba0v9vU1gFxy1Wjoszvl/jbQhmEErtEWCaukx3A==', 'numOfRows': 10, 'pageNo': 1, 'dataType': 'JSON', 'base_date': '20241115', 'base_time': '0500', 'nx': 60, 'ny': 126}

[DEBUG] API 응답 상태: 200
[DEBUG] API 응답 데이터: {'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}, 'body': {'dataType': 'JSON', 'items': {'item': [{'baseDate': '20241115', 'baseTime': '0500', 'category': 'TMP', 'fcstDate': '20241115', 'fcstTime': '0600', 'fcstValue': '14', 'nx': 60, 'ny': 126}, {'baseDate': '20241115', 'baseTime': '0500', 'category': 'UUU', '

'지역: 서울 성동구 옥수동\n 기온: 14°C\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동\n 하늘 상태: 흐림\n 지역: 서울 성동구 옥수동\n 강수 형태: 없음\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동\n 지역: 서울 성동구 옥수동'

In [ ]:
get_weather_response( '대전 유성 원신흥동')

'기온: -999°C\n하늘 상태: 알 수 없음\n강수 형태: 알 수 없음'

In [11]:
from langchain.tools import Tool
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool


import requests
import datetime
import os

from fuzzywuzzy import fuzz
import pandas as pd

# 지역 데이터셋 로드
import pandas as pd
data = pd.read_excel("./data/기상청_단기예보_위경도.xlsx")
# '지역_데이터셋.csv'는 지역명과 좌표 정보를 포함한 파일입니다.

def find_coordinates(location_name):
    best_match = None
    best_score = 0
    latitude = None
    longitude = None

    for idx, row in data.iterrows():
        # 단계별 유사도 비교
        score_1 = fuzz.ratio(location_name, row['1단계']) if pd.notnull(row['1단계']) else 0
        score_2 = fuzz.ratio(location_name, row['2단계']) if pd.notnull(row['2단계']) else 0
        score_3 = fuzz.ratio(location_name, row['3단계']) if pd.notnull(row['3단계']) else 0
        
        # 가중치를 적용한 최종 유사도 계산
        total_score = (score_1 * 0.5) + (score_2 * 0.3) + (score_3 * 0.2)

        # 가장 높은 유사도 점수를 가진 지역 선택
        if total_score > best_score:
            best_score = total_score
            best_match = f"{row['1단계']} {row['2단계']} {str(row['3단계'])}"
            latitude = row['위도']
            longitude = row['경도']

    return {
        '입력 지명': location_name,
        '일치 지명': best_match,
        '일치율': best_score,
        '위도': latitude,
        '경도': longitude
    }



class WeatherFetcher:
    def __init__(self):
        self.base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
        self.api_key = os.getenv('KMA_API_KEY')  # 환경변수에 저장된 기상청 API 인증키
        self.default_x = 55  # 예보지점 X 좌표 (서울)
        self.default_y = 127  # 예보지점 Y 좌표 (서울)

    def fetch_weather(self, date=None, time=None, nx=None, ny=None):
        if not date:
            date = datetime.datetime.now().strftime("%Y%m%d")
        if not time:
            time = "0500"  # 기본 발표시각
        if not nx:
            nx = self.default_x
        if not ny:
            ny = self.default_y

        params = {
            'serviceKey': self.api_key,
            'numOfRows': 10,
            'pageNo': 1,
            'dataType': 'JSON',
            'base_date': date,
            'base_time': time,
            'nx': nx,
            'ny': ny
        }

        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            weather_data = response.json()
            return weather_data
        else:
            print(f"Error: {response.status_code}")
            return None




class WeatherTool:
    def __init__(self):
        self.fetcher = WeatherFetcher()

    def fetch_weather_by_location(self, location_name):
        # 주어진 지역명에 대한 좌표 찾기
        coordinates = find_coordinates(location_name)
        if coordinates['위도'] is not None and coordinates['경도'] is not None:
            # 찾은 좌표를 사용하여 날씨 데이터 가져오기
            weather_data = self.fetcher.fetch_weather(nx=coordinates['경도'], ny=coordinates['위도'])
            return weather_data
        else:
            return None

weather_tool = Tool(
    name="weather_tool",
    description="주어진 지역에 대한 날씨 정보를 가져옵니다.",
    func=WeatherTool().fetch_weather_by_location
)


In [13]:
class WeatherFetcher:
    def __init__(self):
        self.base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
        self.api_key = os.getenv('KMA_API_KEY')  # 환경변수에 저장된 기상청 API 인증키
        self.default_x = 55  # 예보지점 X 좌표 (서울)
        self.default_y = 127  # 예보지점 Y 좌표 (서울)

    def fetch_weather(self, date=None, time=None, nx=None, ny=None):
        if not date:
            date = datetime.datetime.now().strftime("%Y%m%d")
        if not time:
            time = "0500"  # 기본 발표시각
        if not nx:
            nx = self.default_x
        if not ny:
            ny = self.default_y

        params = {
            'serviceKey': self.api_key,
            'numOfRows': 10,
            'pageNo': 1,
            'dataType': 'JSON',
            'base_date': date,
            'base_time': time,
            'nx': nx,
            'ny': ny
        }

        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            weather_data = response.json()
            return weather_data
        else:
            print(f"Error: {response.status_code}")
            return None

class WeatherTool:
    def __init__(self):
        self.fetcher = WeatherFetcher()

    def fetch_weather_by_location(self, location_name):
        coordinates = find_coordinates(location_name)  # find_coordinates 함수는 지역명에 대한 좌표를 반환해야 함
        if coordinates['위도'] is not None and coordinates['경도'] is not None:
            weather_data = self.fetcher.fetch_weather(nx=coordinates['경도'], ny=coordinates['위도'])
            return weather_data
        else:
            return None


In [14]:
def get_weather_response(self, location_name):
    weather_tool = WeatherTool()
    weather_data = weather_tool.fetch_weather_by_location(location_name)
    
    if weather_data:
        items = weather_data['response']['body']['items']['item']
        weather_info = []
        for item in items:
            category = item['category']
            value = item['fcstValue']
            if category == 'TMP':  # 기온
                weather_info.append(f"기온: {value}°C")
            elif category == 'SKY':  # 하늘 상태
                sky_status = {"1": "맑음", "3": "구름많음", "4": "흐림"}.get(value, "알 수 없음")
                weather_info.append(f"하늘 상태: {sky_status}")
            elif category == 'PTY':  # 강수 형태
                rain_status = {"0": "없음", "1": "비", "2": "비/눈", "3": "눈"}.get(value, "알 수 없음")
                weather_info.append(f"강수 형태: {rain_status}")
        return "\n".join(weather_info)
    else:
        return "날씨 정보를 가져오는 데 실패했습니다."


In [ ]:

    def get_ai_response(self, text):
        try:
            self.message_history.append({"role": "user", "content": text})
            if "날씨" in text:
                location_name = re.search(r"날씨\s*([^ ]+)", text)
                if location_name:
                    location_name = location_name.group(1)
                    weather_response = self.get_weather_response(location_name)
                    self.message_history.append({"role": "assistant", "content": weather_response})
                    return weather_response
            
            self.maintain_message_history()
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=self.message_history
            )
            response_text = response.choices[[0]](https://docs.smith.langchain.com/tutorials/Developers/agents).message.content
            self.message_history.append({"role": "assistant", "content": response_text})
            return response_text

        except Exception as e:
            print(f"AI 응답 생성 오류: {e}")
            return "죄송합니다. 응답을 생성하는 중에 오류가 발생했습니다."

SyntaxError: invalid syntax (3914598388.py, line 22)

In [7]:
from langgraph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# Define the state for the graph
class State:
    def __init__(self):
        self.location_name = ""

# Create the graph
builder = StateGraph(State)
builder.add_node("get_weather", weather_tool.fetch_weather_by_location)
builder.add_edge(START, "get_weather")
builder.add_edge("get_weather", END)

# Set up memory for persistence
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

# Function to run the graph
def run_weather_query(location_name):
    state = State()
    state.location_name = location_name
    for event in graph.stream(state, {"configurable": {"thread_id": "1"}}):
        print(event)

# Example usage
run_weather_query("서울")  # Replace with the desired location


ImportError: cannot import name 'StateGraph' from 'langgraph' (unknown location)

In [3]:
import requests
import datetime
import os

class WeatherFetcher:
    def __init__(self):
        self.base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
        self.api_key = os.getenv('KMA_API_KEY')  # 환경변수에 저장된 기상청 API 인증키

    def fetch_weather(self, date=None, time=None, nx=None, ny=None):
        if not date:
            date = datetime.datetime.now().strftime("%Y%m%d")
        if not time:
            time = "0500"  # 기본 발표시각
        if not nx or not ny:
            raise ValueError("좌표(nx, ny)가 필요합니다.")

        params = {
            'serviceKey': self.api_key,
            'numOfRows': 10,
            'pageNo': 1,
            'dataType': 'JSON',
            'base_date': date,
            'base_time': time,
            'nx': nx,
            'ny': ny
        }

        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            weather_data = response.json()
            return weather_data
        else:
            print(f"Error: {response.status_code}")
            return None


In [ ]:
from langchain.tools import tool
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI

# 지역명으로부터 좌표를 찾는 도구 정의
@tool
def get_coordinates(location_name: str) -> str:
    """입력된 지역명에 해당하는 위도와 경도를 반환합니다."""
    result = find_coordinates(location_name)
    if result['위도'] and result['경도']:
        return f"{result['입력 지명']}의 좌표는 위도 {result['위도']}, 경도 {result['경도']}입니다."
    else:
        return "해당 지역의 좌표를 찾을 수 없습니다."

# 날씨 정보를 가져오는 도구 정의
@tool
def get_weather(location_name: str) -> str:
    """입력된 지역명의 현재 날씨 정보를 반환합니다."""
    coordinates = find_coordinates(location_name)
    if coordinates['위도'] and coordinates['경도']:
        fetcher = WeatherFetcher()
        weather_data = fetcher.fetch_weather(nx=coordinates['위도'], ny=coordinates['경도'])
        if weather_data:
            items = weather_data['response']['body']['items']['item']
            weather_info = []
            for item in items:
                category = item['category']
                value = item['fcstValue']
                if category == 'TMP':  # 기온
                    weather_info.append(f"기온: {value}°C")
                elif category == 'SKY':  # 하늘 상태
                    sky_status = {"1": "맑음", "3": "구름많음", "4": "흐림"}.get(value, "알 수 없음")
                    weather_info.append(f"하늘 상태: {sky_status}")
                elif category == 'PTY':  # 강수 형태
                    rain_status = {"0": "없음", "1": "비", "2": "비/눈", "3": "눈"}.get(value, "알 수 없음")
                    weather_info.append(f"강수 형태: {rain_status}")
            return f"{coordinates['입력 지명']}의 현재 날씨 정보:\n" + "\n".join(weather_info)
        else:
            return "날씨 정보를 가져오는 데 실패했습니다."
    else:
        return "해당 지역의 좌표를 찾을 수 없습니다."

# LangChain 에이전트 초기화
tools = [
    Tool(name="Get Coordinates", func=get_coordinates, description="지역명의 좌표를 가져옵니다."),
    Tool(name="Get Weather", func=get_weather, description="지역명의 현재 날씨")]
 


In [5]:
import re

def extract_location(text):
    # 예시 정규식: '서울 날씨 알려줘'에서 '서울' 추출
    match = re.search(r'(\w+)\s*날씨', text)
    if match:
        return match.group(1)
    return None


In [ ]:
from langchain.tools import tool

@tool
def get_coordinates(location_name: str) -> dict:
    """입력된 지역명에 해당하는 위도와 경도를 반환합니다."""
    result = find_coordinates(location_name)
    if result['위도'] and result['경도']:
        return {'latitude': result['위도'], 'longitude': result['경도']}
    else:
        return {'error': '해당 지역의 좌표를 찾을 수 없습니다.'}

@tool
def get_weather_by_coordinates(latitude: float, longitude: float) -> str:
    """입력된 좌표의 현재 날씨 정보를 반환합니다."""
    fetcher = WeatherFetcher()
    weather_data = fetcher.fetch_weather(nx=latitude, ny=longitude)
    if weather_data:
        items = weather_data['response']['body']['items']['item']
        weather_info = []
        for item in items:
            category = item['category']
            value = item['fcstValue']
            if category == 'TMP':  # 기온
                weather_info.append(f"기온: {value}°C")
            elif category == 'SKY':  # 하늘 상태
                sky_status = {"1": "맑음", "3": "구름많음", "4": "흐림"}.get(value, "알 수 없음")
                weather_info.append(f"하늘 상태: {sky_status}")
            elif category == 'PTY':  # 강수 형태
                rain_status = {"0": "없음", "1": "비", "2": "비/눈", "3": "눈"}.get(value, "알 수 없음")
                weather_info.append(f"강수 형태: {rain_status}")
        return "\n".join(weather_info)
    else:
        return "날씨 정보를 가져오는 데 실패했습니다."
